In [596]:
import math
class LKTensor:
    def __init__(self,data,_children=(),_op=''):
        self.data =data
        self.grad=0
        self.leaf=True,
        self._prev=set(_children)
        self._op = _op



    def __repr__(self):
        return 'LKTensor( data:{data}, grad:{grad})'.format(data=self.data,grad=self.grad)

    def __add__(self,other):
        other = other if isinstance(other,LKTensor) else LKTensor(other)
        out=LKTensor(self.data+other.data,(self,other),'+')

        def _backward():
            self.grad+=out.grad
            other.grad+=out.grad
        out._backward=_backward
        out.leaf=False
        return out
    def __mul__(self,other):
        other = other if isinstance(other,LKTensor) else LKTensor(other)
        out=LKTensor(self.data*other.data,(self,other),'*')
        
        def _backward():
            self.grad+=out.grad*other.data
            other.grad+=out.grad*self.data
        out._backward=_backward
        out.leaf=False
        return out
    def __pow__(self,other):
        assert isinstance(other,(int,float)),'only float and int are supported'
        out = LKTensor(self.data**other,(self,),f'**{other}')

        def _backward():
            self.grad+=other*(self.data*(other-1))
        out._backward=_backward
        out.leaf=False
        return out
    
    def tanh(self):
        val = (math.exp(2*self.data)+1)/(math.exp(2*self.data)-1)
        out = LKTensor(val,(self,),'tanh')

        def _backward():
            self.grad+=(1-val**2)*out.grad
        out._backward=_backward
        out.leaf=False
        return out
        
    
    def relu(self):
        out = LKTensor(0 if self.data<0 else self.data,(self,),'relu')
        def _backward():
            self.grad+=(out.data > 0)*out.grad     
        out._backward = _backward
        out.leaf=False

        return out


    def __rpow__(self,other):

        return LKTensor(other**self.data)

    def __neg__(self):
        return self * -1


    def __sub__(self, other):
        return self + (-other)

    def __radd__(self,other):
        return self+other

    def __rmul__(self,other):

        return self*other
    
    def __rsub__(self,other):

        return other + (-self)

    def __truediv__(self, other):
        return self * other**-1

    def __rtruediv__(self, other):
        return other * self**-1

        
    def backward(self):

        topo=[]
        visited = set()

        def build_graph(node):
            if node not in visited:
                visited.add(node)
                
                for child in node._prev:
                    build_graph(child)
                
                topo.append(node)
        
        build_graph(self)
        self.grad=1
        for node in reversed(topo):
            if node.leaf:
                return
            node._backward()

                
                    




    

In [1]:
import random
class Neuron:

    def __init__(self,input_size,non_linear=True):
        self.weights = [LKTensor(random.uniform(-1,1)) for _ in range(input_size)]
        self.bias = LKTensor(0)
        self.non_linear=non_linear

    def __call__(self,input):
        z = sum((wi*xi for wi,xi in zip(self.weights,input)),self.bias)
        self.y = z.tanh() if self.non_linear else z
        return self.y

    def parameters(self):
        return self.weights+[self.bias]

    def zero_grad(self):
        for val in self.parameters():
            val.grad=0
    
    def __repr__(self):

        return f"{'Relu' if self.non_linear else 'Linear'} Neuron ({len(self.weights)})"

    


In [608]:
x=LKTensor(-6)
val = x.tanh()

In [609]:
class Layer:
    def __init__(self,input_size,output_size,**kwargs):
        self.neurons = [Neuron(input_size,**kwargs) for _ in range(output_size)]

    def __call__(self,inputs):
        out = [neuron(inputs) for neuron in self.neurons]
        return out[0] if len(out) == 1 else out
    def parameters(self):
        return [p for neuron in self.neurons for p in neuron.parameters()]

    def __repr__(self):
        return f"Layer of {[', '.join(str(n) for n in self.neurons)]}"



In [610]:
class MLP:

    def __init__(self,input_shape,output_shapes,):
        sizes = [input_shape]+output_shapes
        self.layers = [Layer(sizes[i],sizes[i+1],non_linear=i!=len(output_shapes)-1) for i in range(len(output_shapes))]
    
    def __call__(self,input):
        
        for layer in self.layers:
            input = layer(input)
        return input
    
    def parameters(self):

        return [parameters for layer in self.layers for parameters in layer.parameters()]



In [611]:
X_test = [[2,3,-1],[3,-1,0.5],[0.5,1,1],[1,1,-1]]
Y_test = [1,-1,-1,1]

In [620]:
mlp=MLP(3,[3,4,10,1])

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False


In [621]:
epoch=10

In [622]:
for k in range(20):
    Y_pred = [mlp(x) for x in X_test]
    Loss = sum([(y_pred-y_test)**2 for y_pred,y_test in zip(Y_pred,Y_test)])
    losses[str(k)]=Loss
    for param in mlp.parameters():
        param.grad=0
    Loss.backward()
    for param in mlp.parameters():
        param.data+=(-0.01)*param.grad
    print(k,Loss.data)



0 247.09826528531252
1 318.811544427782
2 318.811544427782
3 96.31330500146564
4 96.31330500146564
5 96.31330500146564
6 61.5946122479764
7 46.982774557575084
8 41.78076131115663
9 76.67453853888613
10 29.670619303137997
11 22.15833657799411
12 22.15833657799411
13 22.15833657799411
14 17.758183922845117
15 17.758183922845117
16 9.511816976158963
17 9.511816976158963
18 6.165823120109595
19 6.165823120109595
